In [20]:
!pip3 install matplotlib
!pip3 install seaborn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.ensemble import AdaBoostRegressor, AdaBoostClassifier
import xgboost as xgb

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
df = pd.read_csv('Pasion et al dataset.csv')

In [4]:
# Encode location data
df_with_location_en = pd.get_dummies(df, columns=['Location'], drop_first=True)

# Encode season data
df_with_loc_season_en = pd.get_dummies(df_with_location_en, columns=['Season'], drop_first=True)

min_hour_of_interest = 10
max_hour_of_interest = 15

df_with_loc_season_en['delta_hr']= df_with_loc_season_en.Hour - min_hour_of_interest

# Create Cyclic date features
df_with_loc_season_en['sine_mon']= np.sin((df_with_loc_season_en.Month - 1)*np.pi/11)
df_with_loc_season_en['cos_mon']= np.cos((df_with_loc_season_en.Month - 1)*np.pi/11)
df_with_loc_season_en['sine_hr']= np.sin((df_with_loc_season_en.delta_hr*np.pi/(max_hour_of_interest - min_hour_of_interest)))
df_with_loc_season_en['cos_hr']= np.cos((df_with_loc_season_en.delta_hr*np.pi/(max_hour_of_interest - min_hour_of_interest)))

In [5]:
selected_columns = ['Latitude', 'Humidity', 'AmbientTemp', 'PolyPwr', 'Wind.Speed',
                     'Visibility', 'Pressure', 'Cloud.Ceiling', 'Location_Grissom',
                     'Location_Hill Weber', 'Location_JDMT', 'Location_Kahului',
                     'Location_MNANG', 'Location_Malmstrom', 'Location_March AFB',
                     'Location_Offutt', 'Location_Peterson', 'Location_Travis',
                     'Location_USAFA','Season_Spring', 'Season_Summer', 'Season_Winter',
                     'sine_mon', 'cos_mon', 'sine_hr', 'cos_hr']

In [6]:
df_processed = df_with_loc_season_en[selected_columns].reset_index(drop=True)

In [7]:
target_label = 'PolyPwr'

In [8]:
input_feat = list(set(selected_columns).difference(set([target_label])))

In [9]:
df_X = df_processed[input_feat].reset_index(drop=True)

In [10]:
df_y = df_processed[target_label]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=42)

In [27]:
#Create lists for hyperparameter tuning

sampling_method_list = ['uniform', 'weighted', 'stratified']
scale_pos_weight_list = [1, 5, 10]                      # Modify the values as needed
alpha_list = [0, 0.1, 0.5, 1]                           # L1 regularization term
lambda_list = [0, 0.1, 0.5, 1]                          # L2 regularization term
min_split_loss_list = [0, 0.1, 0.5, 1]                # Minimum loss reduction required for a split
colsample_bytree_list = np.linspace(0.5, 1.0, 100)      # Subsample ratio of columns when constructing each tree
subsample_list = np.linspace(0.5, 1.0, 100)            # Subsample ratio of the training instances
min_child_weight_list = np.arange(1, 11)                # Minimum sum of instance weight needed in a child
max_depth_list = np.arange(1, 11)                       # Maximum depth of the tree
n_estimators_list = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 20)]          # Number of boosting rounds
learning_rate_list = np.linspace(0.01, 0.3, 100)

In [28]:
xg_base = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

In [29]:
xg_grid = {
    'sampling_method': sampling_method_list,
    'scale_pos_weight': scale_pos_weight_list,
    'alpha': alpha_list,
    'lambda': lambda_list,
    'min_split_loss': min_split_loss_list,
    'colsample_bytree': colsample_bytree_list,
    'subsample': subsample_list,
    'min_child_weight': min_child_weight_list,
    'max_depth': max_depth_list,
    'n_estimators': n_estimators_list,
    'learning_rate': learning_rate_list,
                }

In [30]:
xg_random = RandomizedSearchCV(estimator=xg_base, param_distributions=xg_grid, 
                                 n_iter=1000, cv=4, verbose=2, random_state=42, 
                                 n_jobs=-1)

In [32]:
xg_random.fit(X_train, y_train)

Fitting 4 folds for each of 1000 candidates, totalling 4000 fits
[CV] END alpha=0.5, colsample_bytree=0.5202020202020202, lambda=0.5, learning_rate=0.23262626262626263, max_depth=10, min_child_weight=3, min_split_loss=0, n_estimators=400, sampling_method=weighted, scale_pos_weight=10, subsample=0.8636363636363636; total time=   0.0s
[CV] END alpha=0.5, colsample_bytree=0.5202020202020202, lambda=0.5, learning_rate=0.23262626262626263, max_depth=10, min_child_weight=3, min_split_loss=0, n_estimators=400, sampling_method=weighted, scale_pos_weight=10, subsample=0.8636363636363636; total time=   0.0s
[CV] END alpha=0.5, colsample_bytree=0.5202020202020202, lambda=0.5, learning_rate=0.23262626262626263, max_depth=10, min_child_weight=3, min_split_loss=0, n_estimators=400, sampling_method=weighted, scale_pos_weight=10, subsample=0.8636363636363636; total time=   0.0s
[CV] END alpha=0.1, colsample_bytree=0.5757575757575758, lambda=1, learning_rate=0.03636363636363636, max_depth=4, min_child_

/Users/rishisankhe/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
2680 fits failed out of a total of 4000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1424 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/rishisankhe/Library/Python/3.9/lib/python/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/rishisankhe/Library/Python/3.9/lib/python/site-packages/xgboost/core.py", line 620, in inner_f
    return func(**kwargs)
  File "/Users/rishisankhe/Library/Python/3.9/lib/python/site-packages/xgboost/sklearn.py", line 1025, in fit
    self._Booster 

RandomizedSearchCV(cv=4,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, gpu_id=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
       0.82828283, 0.83333333, 0.83838384, 0.84343434, 0.84848485,
       0.85353535, 0.85858586, 0.86363636, 0.86868687, 0.87373737,
       0.87878788, 0.88383838, 0.88888889, 0.89393939, 0.8989899 ,
       0.9040404 , 0.90909091, 0.91414141, 0.91919192, 0.92424242,
       0.92929293, 0.93434343, 0.93939394, 0.94444444, 0.94949495,
       0.95454545, 0.95959596, 0.96464646, 0.96969697, 0.97474747,
       0.97979798, 0.98484848, 0.98989899, 0.99494949, 1.        ])},
                   random_state=42, verbose=2)

[CV] END alpha=0, colsample_bytree=0.9141414141414141, lambda=0.1, learning_rate=0.2619191919191919, max_depth=2, min_child_weight=6, min_split_loss=0.5, n_estimators=1800, sampling_method=uniform, scale_pos_weight=5, subsample=0.8535353535353536; total time=  14.7s
[CV] END alpha=0.1, colsample_bytree=0.6111111111111112, lambda=0.5, learning_rate=0.05686868686868687, max_depth=8, min_child_weight=2, min_split_loss=0, n_estimators=700, sampling_method=weighted, scale_pos_weight=5, subsample=0.6616161616161617; total time=   0.0s
[CV] END alpha=0.1, colsample_bytree=0.7323232323232324, lambda=0.1, learning_rate=0.018787878787878787, max_depth=8, min_child_weight=10, min_split_loss=0.5, n_estimators=500, sampling_method=uniform, scale_pos_weight=1, subsample=0.6818181818181819; total time=  14.4s
[CV] END alpha=0.5, colsample_bytree=0.9141414141414141, lambda=0.1, learning_rate=0.01292929292929293, max_depth=3, min_child_weight=1, min_split_loss=0, n_estimators=1100, sampling_method=unif

In [33]:
xg_random.best_params_

{'subsample': 0.6818181818181819,
 'scale_pos_weight': 1,
 'sampling_method': 'uniform',
 'n_estimators': 500,
 'min_split_loss': 0.5,
 'min_child_weight': 10,
 'max_depth': 8,
 'learning_rate': 0.018787878787878787,
 'lambda': 0.1,
 'colsample_bytree': 0.7323232323232324,
 'alpha': 0.1}

In [34]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(xg_random.predict(X_test), y_test)

2.7454971636225234

In [36]:
from sklearn.metrics import mean_squared_error
mean_squared_error(xg_random.predict(X_test), y_test) #square root of 16.516 is 4.06 which is the third best model

16.516109238578107

In [39]:
xg_random.best_score_

0.6718798287309605

In [40]:
#Training model using optimal hyperparameters
xg_model = xgb.XGBRegressor(**xg_random.best_params_, random_state=42)

In [41]:
xg_model.fit(X_train,y_train)

XGBRegressor(alpha=0.1, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7323232323232324, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=0.1,
             learning_rate=0.018787878787878787, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=10, min_split_loss=0.5, missing=nan,
             monotone_constraints=None, n_estimators=500, n_jobs=None, ...)

In [42]:
y_pred = xg_model.predict(X_test)

In [44]:
# R2 score
r2_score(y_test.ravel(), y_pred)

0.6748829875805387

In [45]:
# Mean absolute error
mean_absolute_error(y_test.ravel(), y_pred)

2.7454971636225234

In [46]:
# Root mean square error
np.sqrt(mean_squared_error(y_test.ravel(), y_pred))

4.064001628761744